#TP2 - Spark

S12. Top 10 películas Argentinas según vote_average que contengan el género drama con más de 100 reviews en ratings.csv (dar géneros, cantidad de ratings, título, y vote_average).

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [10]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

pelis = sqlContext.read.parquet('/content/drive/MyDrive/Organización de Datos/TP2/Movie dataset/movies.parquet').rdd
ratings = sqlContext.read.parquet('/content/drive/MyDrive/Organización de Datos/TP2/Movie dataset/ratings.parquet').rdd

In [29]:
pelis_drama_arg = pelis.filter(lambda x: x.genres != None and x.production_countries != None)
pelis_drama_arg = pelis_drama_arg.filter(lambda x: "Drama" in x.genres and "Argentina" in x.production_countries).map(lambda x: (int(x.id), [x.title, x.genres, x.vote_average]))

ratings_100 = ratings.map(lambda x: (x.movieId, 1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] > 100)

pelis_drama_arg_100ratings = pelis_drama_arg.join(ratings_100).map(lambda x: (x[0], x[1][0] + [x[1][1]]))
pelis_drama_arg_100ratings.takeOrdered(10, lambda x: -x[1][2])

[(16, ['Dancer in the Dark', 'Drama,Crime,Music', 7.7, 20517]),
 (1653, ['The Motorcycle Diaries', 'Drama', 7.2, 23335]),
 (5333, ['Apartment Zero', 'Thriller,Horror,Drama', 6.9, 385]),
 (47261, ['Carancho', 'Thriller,Drama,Romance', 6.5, 234]),
 (48596, ['Ana and the Others', 'Drama,Foreign', 6.3, 273]),
 (1956, ['Gerry', 'Mystery,Drama,Adventure', 6.1, 4707]),
 (45722, ["Don't Look Down", 'Drama,Romance', 5.6, 15079]),
 (6636, ['The Mugger', 'Drama,Foreign', 5.1, 974])]